In [1]:
%load_ext ngql

In [8]:
import os
import logging
import sys

import openai
import pandas as pd
from dotenv import load_dotenv
from IPython.display import Markdown, display
from llama_index.core import load_index_from_storage
from llama_index.core import (
    Document,
    KnowledgeGraphIndex,
    StorageContext,
)
from llama_index.core.query_engine import KnowledgeGraphQueryEngine
from llama_index.core.settings import Settings
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.llms.openai import OpenAI
from llama_index.graph_stores.nebula import NebulaGraphStore
from llama_index.legacy.query_engine.knowledge_graph_query_engine import DEFAULT_NEO4J_NL2CYPHER_PROMPT


load_dotenv()

logging.basicConfig(
    stream=sys.stdout,
    level=logging.DEBUG,
)

openai.api_key = os.environ["OPENAI_API_KEY"]


In [12]:
space_name = "air_france"
edge_types, rel_prop_names = ["relationship"], [
    "name"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)

storage_context = StorageContext.from_defaults(
    persist_dir="../../data/index/policies/air_france",
    graph_store=graph_store
)

kg_index = load_index_from_storage(
    storage_context=storage_context,
    max_triplets_per_chunk=20,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    verbose=True,
)

DEBUG:llama_index.core.storage.kvstore.simple_kvstore:Loading llama_index.core.storage.kvstore.simple_kvstore from ../../data/index/policies/air_france/docstore.json.
DEBUG:fsspec.local:open file: /Users/pez/Projects/nlp_airline_project/notebooks/kg-llamaindex/../../data/index/policies/air_france/docstore.json
DEBUG:llama_index.core.storage.kvstore.simple_kvstore:Loading llama_index.core.storage.kvstore.simple_kvstore from ../../data/index/policies/air_france/index_store.json.
DEBUG:fsspec.local:open file: /Users/pez/Projects/nlp_airline_project/notebooks/kg-llamaindex/../../data/index/policies/air_france/index_store.json
DEBUG:llama_index.core.vector_stores.simple:Loading llama_index.core.vector_stores.simple from ../../data/index/policies/air_france/image__vector_store.json.
DEBUG:fsspec.local:open file: /Users/pez/Projects/nlp_airline_project/notebooks/kg-llamaindex/../../data/index/policies/air_france/image__vector_store.json
DEBUG:llama_index.core.vector_stores.simple:Loading llam

In [13]:
llm = OpenAI(temperature=0, model="gpt-3.5-turbo") #  model="gpt-3.5-turbo")

nl2kg_query_engine = KnowledgeGraphQueryEngine(
    graph_query_synthesis_prompt=DEFAULT_NEO4J_NL2CYPHER_PROMPT,
    llm=llm,
    refresh_schema=True,
    storage_context=storage_context,
    verbose=True,
)

DEBUG:llama_index.graph_stores.nebula.base:get_schema()
schema: Node properties: [{'tag': 'entity', 'properties': [('name', 'string')]}]
Edge properties: [{'edge': 'relationship', 'properties': [('name', 'string')]}]
Relationships: ['(:entity)-[:relationship]->(:entity)']



In [15]:
retriever_graph_rag = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=True,
)

query_engine_graph_rag = RetrieverQueryEngine.from_args(retriever_graph_rag)

retriever_graph_rag_with_nl2graphquery = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=True,
    with_nl2graphquery=True,
)

query_engine_with_nl2graphquery = RetrieverQueryEngine.from_args(retriever_graph_rag_with_nl2graphquery)

In [23]:
query_engine = kg_index.as_query_engine(
    verbose=True,
)

In [24]:
question = """
Tell me about check bag fees.
"""

response_nl2kg = query_engine.query(question)

# Cypher:

print("The Cypher Query is:")

# query_string = query_engine.generate_query(question)
#
#display(
#    Markdown(
#        f"""
#```cypher
#{query_string}
#```
#"""
#    )
#)
#
#%ngql policies_aa {query_string}

# Answer:

print("The Answer is:")

display(Markdown(f"<b>{response_nl2kg}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\n\nTell me about check bag fees.\n\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None


DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x2b1f80810>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x2b11b7da0> server_hostname='api.openai.com' timeout=60.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x2a38c2b10>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sat, 11 May 2024 21:16:51 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'user-hrq

<b>Baggage checks will be enforced prior to boarding. If your hand baggage does not meet the specified criteria, you may be asked to place it in the hold and could incur additional fees.</b>

In [25]:
kg_index_query_engine = kg_index.as_query_engine(
    retriever_mode="keyword",
    verbose=True,
    response_mode="tree_summarize",
)

In [26]:
response_graph_rag = kg_index_query_engine.query("Tell me about baggage policies on flights to Cuba")

display(Markdown(f"<b>{response_graph_rag}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nTell me about baggage policies on flights to Cuba\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.Sy

DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sat, 11 May 2024 21:17:18 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'user-hrqyc9oyfn1jyrx5n6yy9lct'), (b'openai-processing-ms', b'308'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'60000'), (b'x-ratelimit-remaining-requests', b'7976'), (b'x-ratelimit-remaining-tokens', b'59890'), (b'x-ratelimit-reset-requests', b'4h51m20.041s'), (b'x-ratelimit-reset-tokens', b'110ms'), (b'x-request-id', b'req_1a60ecad98c1af3ee980ca2ee9757329'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'882531eaa8112a

<b>Baggage policies on flights to Cuba typically adhere to standard international airline regulations regarding the number of bags allowed, weight limits, and any additional fees for excess baggage. It is advisable to check with the specific airline you are flying with to Cuba for the most accurate and up-to-date information on their baggage policies.</b>

In [27]:
response = kg_index.as_retriever().retrieve("Tell me about baggage policies on flights to Cuba")
display(Markdown(f"<b>{response}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nTell me about baggage policies on flights to Cuba\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.Sy

<b>[NodeWithScore(node=TextNode(id_='ca266b69-3a25-451b-9fd4-d76f5e119c79', embedding=None, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='policy_178', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='a40f20e5e7092cc78925b6c878ba0709734c245b85eae96a5acf4ec4db6817ed'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='937ffc32-1b37-4a2f-a09b-257ac7786319', node_type=<ObjectType.TEXT: '1'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='661756255a1579964307fc29c461f83109031464b1af5152a9681bde3efe2292'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='5e947970-fd6c-48a5-989c-dc61796924f3', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='44ec5151c78ce8c985bc405fdf28e45a3ae71b9ca0c17569e195aad244f5ed1e')}, text='(including pockets, wheels, and handles). Not sure if your bag meets these dimension requirements? Air\xa0France offers a range of tools for you to check your baggage dimensions. You will find them at the airport, baggage drop-off areas, and in check-in halls. Baggage checks will be enforced prior to boarding. If your hand baggage does not meet these criteria, you will be asked to place it in the hold. You may incur additional fees.  \nYour hand baggage weight  \nIn the Economy or Premium Economy cabins, your hand baggage and personal items can weigh up to 12 kg (26.4 lb) in total. If you are traveling in the Business cabin or in La Première, your hand baggage and personal items can weigh up to 18 kg (40 lb) in total. Your cabin is shown on your ticket.', start_char_idx=1362, end_char_idx=2120, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1000.0), NodeWithScore(node=TextNode(id_='937ffc32-1b37-4a2f-a09b-257ac7786319', embedding=None, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='policy_178', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='a40f20e5e7092cc78925b6c878ba0709734c245b85eae96a5acf4ec4db6817ed'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='7d5614b5-fb24-47b2-86ef-cf8fd2fbcf54', node_type=<ObjectType.TEXT: '1'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='e62815d128ddd07b0e65ae88db5f2c10cd1d3fb4a4d5d3c22bdd744541516735'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='ca266b69-3a25-451b-9fd4-d76f5e119c79', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='9fa9fce6791d358eca3727e60e064bdd89aec5e22b2db699418e723f6393b80c')}, text='You may incur additional fees.\nHand baggage dimensions\nThe maximum dimensions of your hand baggage are: 55\xa0x\xa035\xa0x\xa025\xa0cm/21.7 x 13.8 x 9.9 in. (including pockets, wheels, and handles). Not sure if your bag meets these dimension requirements? Air\xa0France offers a range of tools for you to check your baggage dimensions. You will find them at the airport, baggage drop-off areas, and in check-in halls. Baggage checks will be enforced prior to boarding. If your hand baggage does not meet these criteria, you will be asked to place it in the hold. You may incur additional fees.  \nThe maximum dimensions of your hand baggage are: 55\xa0x\xa035\xa0x\xa025\xa0cm/21.7 x 13.8 x 9.9 in.', start_char_idx=38158, end_char_idx=38822, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1000.0), NodeWithScore(node=TextNode(id_='093f0301-9688-4f96-abf5-ec00aad1fdde', embedding=None, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='policy_178', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='a40f20e5e7092cc78925b6c878ba0709734c245b85eae96a5acf4ec4db6817ed'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='c538533f-8370-4e6a-ba1f-f36f391666fc', node_type=<ObjectType.TEXT: '1'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='7971b19771768641f24eb73d8755902824fd9ae36db3259d2e49e0f4fcd8d774'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='7d5614b5-fb24-47b2-86ef-cf8fd2fbcf54', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='602f856d666d63f6997bc1f783468a85c7b68c142088ad00652d0cda46ca70ba')}, text='Baggage checks will be enforced prior to boarding. If your hand baggage does not meet these criteria, you will be asked to place it in the hold. You may incur additional fees.  \nHand baggage dimensions  \nThe maximum dimensions of your hand baggage are: 55\xa0x\xa035\xa0x\xa025\xa0cm/21.7 x 13.8 x 9.9 in. (including pockets, wheels, and handles). Not sure if your bag meets these dimension requirements? Air\xa0France offers a range of tools for you to check your baggage dimensions. You will find them at the airport, baggage drop-off areas, and in check-in halls. Baggage checks will be enforced prior to boarding. If your hand baggage does not meet these criteria, you will be asked to place it in the hold. You may incur additional fees.', start_char_idx=36915, end_char_idx=37639, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1000.0), NodeWithScore(node=TextNode(id_='9ebaa8fb-fa4c-4d23-9bdd-5f546b37c4cc', embedding=None, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='policy_177', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='3e193d65d9827fcc5a324c6122b566778f2468de4b7059014ef8608435467eda'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='cac63e7b-2b42-41da-87ed-9c31d8113a80', node_type=<ObjectType.TEXT: '1'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='cfb450c5ef735b25141279d63d705071a27b159bb2bb9d7b129c09203662ef16'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='211c7f01-562d-4833-9236-88d86150f7f0', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='4139c73e12646e0ae5f03f1c4df7a45efbf2b42ceac3c3369ee551a57a0d639b')}, text='Air France Customer Service  \nDo you need further assistance with your missing baggage claim?  \nMissing baggage  \nDamaged bagggage - Missing item  \nMissing baggage  \nDamaged bagggage - Missing item  \nMissing baggage  \nDamaged bagggage - Missing item  \nMissing baggage  \nDamaged bagggage - Missing item  \nMissing baggage  \nDamaged bagggage - Missing item  \nDeclare a missing baggage item onlineDeclare a missing baggage item at the airportHow can I track the status of my missing baggage claim?How can I make a claim for my basic necessities?Can I come to the airport if my tracking device shows my baggage is there?What should I do if my baggage has not been found within 72 hours of submitting my claim?', start_char_idx=9136, end_char_idx=9840, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1000.0), NodeWithScore(node=TextNode(id_='d9bc6644-ca48-42c1-84ed-f4b691951f39', embedding=None, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='policy_177', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='3e193d65d9827fcc5a324c6122b566778f2468de4b7059014ef8608435467eda'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='09f9a570-ff58-49d0-9936-9ce061918749', node_type=<ObjectType.TEXT: '1'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='8dd836e2b3177b4b06ecea3dc3a18064579afe85536726a9da5d3224844f5506'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='f514a2d6-8692-4213-be03-a88ad2f3fd92', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1d712b0d7e239ff0452ecab34e2c4087b093a7f7c1c1128f39874eab4c4bf07e')}, text='The more detail you provide, the better we will be able to locate and identify your baggage. If your baggage has not been found within 21 days of \xa0your request, the items you have inventoried will serve as the basis of your compensation.*\xa0 If you can produce receipts (invoices, proof of purchase), your allowance can be as much as €1,607 (the maximum amount set by the Montreal convention). Please note\xa0: when you check in your baggage, you can purchase additional insurance that allows you to declare the actual value of your belongings, if this is estimated to be higher than the maximum compensation the Montreal Convention provides (€1,607*). Therefore, the refund will be calculated according to the declared value if your baggage is lost.', start_char_idx=3430, end_char_idx=4175, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1000.0), NodeWithScore(node=TextNode(id_='46acb5ca-e3b8-4b03-b9a2-e18579014d73', embedding=None, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='policy_177', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='3e193d65d9827fcc5a324c6122b566778f2468de4b7059014ef8608435467eda'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='08c3a380-144a-4c54-8d88-8a14e4059955', node_type=<ObjectType.TEXT: '1'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='0acd07de75f11fb6eb364ec29150e113d68be860b6b798a9f0e78a2f6fd89796'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='20677100-f45a-41a0-a958-22f7f7ab0030', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='a70564fa81687cd6495a774b757415d1c72573f874bf4b2c69827c745f5e6737')}, text='Make sure to keep all your receipts, as you may be asked to submit them.  \nAccess the claim form online  \nAccess the claim form online  \nAccess the claim form online  \nAccess the claim form online  \nAlthough your baggage has a tracking device, we kindly ask that you not come to the airport. Your tracking device (e.g., AirTag or SmartTag) might tell you the whereabouts of your baggage, but it is not linked to our systems. When your baggage has arrived at the destination airport, internal processes and customs will still require some time before it can be delivered to you or until it is ready for collection. You will be notified once your baggage is ready to be picked up or delivered.\xa0Please include your contact details with your tracking device so we can reach you if necessary.  \nAlthough your baggage has a tracking device, we kindly ask that you not come to the airport.', start_char_idx=56347, end_char_idx=57229, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1000.0), NodeWithScore(node=TextNode(id_='91cf7177-b409-4a07-8a40-ddcc54b4e80c', embedding=None, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='policy_177', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='3e193d65d9827fcc5a324c6122b566778f2468de4b7059014ef8608435467eda'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='73739f86-7a5b-4e37-9f25-986bcd781bd3', node_type=<ObjectType.TEXT: '1'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='5496fcdee2988a51abc303ff55ef601e386997a5a4a64b45e6507aa355ce1e0f'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='b9f08d5f-66d5-4aff-a22f-d22d00233c1f', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='d98eb30d88c6865a9a8ad2aebff909ce0e4bcab562e48ab1beb1288bb46c53a7')}, text='To do so, please locate your baggage receipt. Where can you find my baggage receipt?\xa0 - If you checked in at the airport, your receipt was given \xa0to you by \xa0one of our agents at the drop-off area. - If you printed out your baggage tags and boarding pass at home, your receipt number will be located on the Baggage \xa0Receipt page \xa0and on the confirmation email that was sent to you. Add the receipt number to the online form. You will receive a \xa0claim number once you have submitted your request. Keep your claim number close to track the status of your request. Please note that certain airports do not allow you to declare a missing baggage item online.', start_char_idx=797, end_char_idx=1450, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1000.0), NodeWithScore(node=TextNode(id_='60b7a36c-4623-4b06-883e-5223c1fc6dbd', embedding=None, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='policy_177', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='3e193d65d9827fcc5a324c6122b566778f2468de4b7059014ef8608435467eda'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='b9f08d5f-66d5-4aff-a22f-d22d00233c1f', node_type=<ObjectType.TEXT: '1'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='8c311c3d97d95b5746242fab867f16260a7fde625ea3ed745880971418d6cdf5'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='2978e194-79be-4dab-aab8-97e429695796', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='07e583d14cb1b77e8172150beb599042183630aec94d7e3c69147ab77d39402e')}, text='View your request  \nYou have incurred basic expenses (e.g. purchase of hygiene products or clothing) due to the absence of your baggage\xa0? You \xa0have 21 days to submit your reimbursement request from the \xa0day you receive your baggage. Please fill out the reimbursement form online. Make sure to keep all your receipts, as you may be asked to submit them.  \nAccess the claim form online  \nAlthough your baggage has a tracking device, we kindly ask that you not come to the airport. Your tracking device (e.g., AirTag or SmartTag) might tell you the whereabouts of your baggage, but it is not linked to our systems. When your baggage has arrived at the destination airport, internal processes and customs will still require some time before it can be delivered to you or until it is ready for collection.', start_char_idx=2171, end_char_idx=2971, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1000.0), NodeWithScore(node=TextNode(id_='3dfe4612-b96b-494d-a25c-c9df6fc40c09', embedding=None, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='policy_177', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='3e193d65d9827fcc5a324c6122b566778f2468de4b7059014ef8608435467eda'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='2f18d8a6-91d2-4fd7-961b-3277ec97283d', node_type=<ObjectType.TEXT: '1'>, metadata={'airline': 'Air France', 'policy': 'Restricted Items Policy', 'topic': nan}, hash='3029289fb49aad66b439b969eb0858dce876a245256104a5ced7c9d262096ea4'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='8f84229e-4c77-4e3d-86d8-e00dddc70842', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='57638b098040f54be5f73f5205114f230a101988a3063b8a8d8b189069dfae3b')}, text='Please note\xa0: when you check in your baggage, you can purchase additional insurance that allows you to declare the actual value of your belongings, if this is estimated to be higher than the maximum compensation the Montreal Convention provides (€1,607*). Therefore, the refund will be calculated according to the declared value if your baggage is lost. * Compensation is expressed in Special Drawing Rights (SDRs), the unit of account of the International\xa0Monetary\xa0Fund (IMF)\xa0: 1\xa0288\xa0SDR equals approximately €1,\xa0607\xa0. This amount is subject to currency valuations and is not guaranteed.  \nOpen all  \nUsing our online baggage tracker, you can track your missing baggage claim in real-time. To do so, please use the claim number that was given to you when you submitted your online form, or by one of our agents at the airport.', start_char_idx=46658, end_char_idx=47485, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1000.0), NodeWithScore(node=TextNode(id_='3b2836d4-0fd3-44f1-aab1-59bee1e15d82', embedding=None, metadata={'kg_rel_texts': ["('Baggage checks', 'Will be enforced', 'Prior to boarding')", "('Baggage', 'Can', 'Declare at the airport')"], 'kg_rel_map': {}, 'kg_schema': {'schema': "Node properties: [{'tag': 'entity', 'properties': [('name', 'string')]}]\nEdge properties: [{'edge': 'relationship', 'properties': [('name', 'string')]}]\nRelationships: ['(:entity)-[:relationship]->(:entity)']\n"}}, excluded_embed_metadata_keys=['kg_rel_map', 'kg_rel_texts'], excluded_llm_metadata_keys=['kg_rel_map', 'kg_rel_texts'], relationships={}, text="The following are knowledge sequence in max depth 2 in the form of directed graph like:\n`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`\n('Baggage checks', 'Will be enforced', 'Prior to boarding')\n('Baggage', 'Can', 'Declare at the airport')", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1000.0)]</b>

In [28]:
from pprint import pprint

for node in response:
    print("NODE:")
    pprint(node.dict())
    if hasattr(node, "relationships"):
        for key, rel in node.relationships.items():
            print("RELATION: ", key)
            pprint(rel.dict())


NODE:
{'class_name': 'NodeWithScore',
 'node': {'class_name': 'TextNode',
          'embedding': None,
          'end_char_idx': 2120,
          'excluded_embed_metadata_keys': [],
          'excluded_llm_metadata_keys': [],
          'id_': 'ca266b69-3a25-451b-9fd4-d76f5e119c79',
          'metadata': {'airline': 'Air France',
                       'policy': 'Restricted Items Policy',
                       'topic': nan},
          'metadata_seperator': '\n',
          'metadata_template': '{key}: {value}',
          'relationships': {<NodeRelationship.SOURCE: '1'>: {'class_name': 'RelatedNodeInfo',
                                                             'hash': 'a40f20e5e7092cc78925b6c878ba0709734c245b85eae96a5acf4ec4db6817ed',
                                                             'metadata': {'airline': 'Air '
                                                                                     'France',
                                                                   

-----

In [29]:
question = "What is the baggage fee for economy class on a flight to Cuba?"
response_graph_rag = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q:What is the baggage fee for economy class on a flight to Cuba?<b>\n\n<b>A: {response_graph_rag}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the baggage fee for economy class on a flight to Cuba?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._bac

<b>Q:What is the baggage fee for economy class on a flight to Cuba?<b>

<b>A: The baggage fee for economy class on a flight to Cuba varies depending on the airline and the specific fare class purchased. It is recommended to check with the airline directly or visit their website for the most up-to-date information on baggage fees for flights to Cuba.</b>

In [30]:
question = "What policies does American Airlines have?"
response_graph_rag = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {response_graph_rag}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat policies does American Airlines have?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStrea

<b>Q: What policies does American Airlines have?<b>

<b>A: American Airlines has various policies in place regarding baggage, check-in procedures, flight changes, cancellations, and onboard services. These policies are designed to ensure smooth operations and provide a positive experience for passengers.</b>

In [31]:
question = "What is not allowed on American Airlines?"
response_graph_rag = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q:What is the baggage fee for economy class on a flight to Cuba?<b>\n\n<b>A: {response_graph_rag}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is not allowed on American Airlines?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream

<b>Q:What is the baggage fee for economy class on a flight to Cuba?<b>

<b>A: Pets in the cabin are not allowed on American Airlines.</b>

In [32]:
question = "Are batteries allowed on flights?"
response_graph_rag = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q:What is the baggage fee for economy class on a flight to Cuba?<b>\n\n<b>A: {response_graph_rag}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nAre batteries allowed on flights?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object 

<b>Q:What is the baggage fee for economy class on a flight to Cuba?<b>

<b>A: Yes, batteries are generally allowed on flights, but there are restrictions and guidelines to follow depending on the type and size of the battery.</b>

In [33]:
question = "Are batteries allowed on flights?"
response = kg_index.as_retriever().retrieve(question)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nAre batteries allowed on flights?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None


DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x2b7815550>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x2b11b7da0> server_hostname='api.openai.com' timeout=60.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x2b1f9d150>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sat, 11 May 2024 21:21:13 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'user-hrq

In [34]:
for node in response:
    print("NODE:")
    pprint(node.dict())
    if hasattr(node, "relationships"):
        for key, rel in node.relationships.items():
            print("RELATION: ", key)
            pprint(rel.dict())


NODE:
{'class_name': 'NodeWithScore',
 'node': {'class_name': 'TextNode',
          'embedding': None,
          'end_char_idx': 4166,
          'excluded_embed_metadata_keys': [],
          'excluded_llm_metadata_keys': [],
          'id_': '207ecf5e-0144-4332-84fe-e1a6c2dc2f1d',
          'metadata': {'airline': 'Air France',
                       'policy': 'Restricted Items Policy',
                       'topic': nan},
          'metadata_seperator': '\n',
          'metadata_template': '{key}: {value}',
          'relationships': {<NodeRelationship.SOURCE: '1'>: {'class_name': 'RelatedNodeInfo',
                                                             'hash': 'a40f20e5e7092cc78925b6c878ba0709734c245b85eae96a5acf4ec4db6817ed',
                                                             'metadata': {'airline': 'Air '
                                                                                     'France',
                                                                   

In [35]:
question = "What is the baggage fee for economy class for American Airline?"
query_response = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {query_response}</b>"))

nodes = kg_index.as_retriever().retrieve(question)
for node in nodes:
    print("NODE:")
    pprint(node.dict())
    if hasattr(node, "relationships"):
        for key, rel in node.relationships.items():
            print("RELATION: ", key)
            pprint(rel.dict())


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the baggage fee for economy class for American Airline?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._ba

<b>Q: What is the baggage fee for economy class for American Airline?<b>

<b>A: The baggage fee for economy class for American Airline is not provided in the given context information.</b>

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the baggage fee for economy class for American Airline?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:recei

In [36]:
question = "What is the bag fee for checked bag for American Airline?"
query_response = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {query_response}</b>"))

nodes = kg_index.as_retriever().retrieve(question)
for node in nodes:
    print("NODE:")
    pprint(node.dict())
    if hasattr(node, "relationships"):
        for key, rel in node.relationships.items():
            print("RELATION: ", key)
            pprint(rel.dict())


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the bag fee for checked bag for American Airline?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_res

<b>Q: What is the bag fee for checked bag for American Airline?<b>

<b>A: There is no information provided in the context regarding the bag fee for checked bags for American Airlines.</b>

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the bag fee for checked bag for American Airline?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_res

In [37]:
nl2kg_query_engine = KnowledgeGraphQueryEngine(
    graph_query_synthesis_prompt=DEFAULT_NEO4J_NL2CYPHER_PROMPT,
    llm=llm,
    refresh_schema=True,
    storage_context=storage_context,
    verbose=True,
    graph_store_query_depth=3,
)

kg_index_query_engine = kg_index.as_query_engine(
    retriever_mode="hybrid",
    verbose=True,
    response_mode="tree_summarize",
    graph_store_query_depth=4,
)

question = "What is the bag fee for checked bag for American Airline?"
query_response = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {query_response}</b>"))

if False:
    nodes = kg_index.as_retriever().retrieve(question)
    for node in nodes:
        print("NODE:")
        pprint(node.dict())
        if hasattr(node, "relationships"):
            for key, rel in node.relationships.items():
                print("RELATION: ", key)
                pprint(rel.dict())


DEBUG:llama_index.graph_stores.nebula.base:get_schema()
schema: Node properties: [{'tag': 'entity', 'properties': [('name', 'string')]}]
Edge properties: [{'edge': 'relationship', 'properties': [('name', 'string')]}]
Relationships: ['(:entity)-[:relationship]->(:entity)']

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the bag fee for checked bag for American Airline?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.

<b>Q: What is the bag fee for checked bag for American Airline?<b>

<b>A: There is no information provided in the context about the bag fee for checked bags for American Airlines.</b>

In [38]:

kg_index_query_engine = kg_index.as_query_engine(
    retriever_mode="hybrid",
    verbose=True,
    response_mode="tree_summarize",
    graph_store_query_depth=4,
)

question = "What is the bag fee for checked bag for American Airline?"
query_response = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {query_response}</b>"))


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the bag fee for checked bag for American Airline?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_res

DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sat, 11 May 2024 21:22:47 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'user-hrqyc9oyfn1jyrx5n6yy9lct'), (b'openai-processing-ms', b'227'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'60000'), (b'x-ratelimit-remaining-requests', b'7897'), (b'x-ratelimit-remaining-tokens', b'58304'), (b'x-ratelimit-reset-requests', b'5h2m41.622s'), (b'x-ratelimit-reset-tokens', b'1.695s'), (b'x-request-id', b'req_a79f85b94928153b2b56913cae9565c3'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'882539f4eb77111e-ORD'), (b'Content-Encoding', b'gzip'), (b'alt-svc', b'h3=":443"; ma=86400')])
INFO:httpx:HTTP Request: POST https://api.openai.co

<b>Q: What is the bag fee for checked bag for American Airline?<b>

<b>A: There is no information provided in the context about the bag fee for checked bags for American Airlines.</b>

In [39]:

question = "What is the fee for checked bag for American Airlines?"
query_response = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {query_response}</b>"))

if False:
    nodes = kg_index.as_retriever().retrieve(question)
    for node in nodes:
        print("NODE:")
        pprint(node.dict())
        if hasattr(node, "relationships"):
            for key, rel in node.relationships.items():
                print("RELATION: ", key)
                pprint(rel.dict())


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the fee for checked bag for American Airlines?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sy

DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sat, 11 May 2024 21:23:21 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'user-hrqyc9oyfn1jyrx5n6yy9lct'), (b'openai-processing-ms', b'232'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'60000'), (b'x-ratelimit-remaining-requests', b'7890'), (b'x-ratelimit-remaining-tokens', b'59888'), (b'x-ratelimit-reset-requests', b'5h3m42.956s'), (b'x-ratelimit-reset-tokens', b'112ms'), (b'x-request-id', b'req

<b>Q: What is the fee for checked bag for American Airlines?<b>

<b>A: The fee for checked bags on Air France may vary depending on the specific circumstances of the flight. In some cases, passengers may be asked to check in their hand baggage free of charge, especially if the flight is full or depending on the type of aircraft. However, if hand baggage does not meet the specified criteria or dimensions, passengers may incur additional fees.</b>

In [70]:
question = "What is the fee for checked bag for American Airlines?"
query_response = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {query_response}</b>"))


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the fee for checked bag for American Airlines?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sy

<b>Q: What is the fee for checked bag for American Airlines?<b>

<b>A: The fee for a checked bag on American Airlines varies depending on the specific fare type and route. It is recommended to check the most up-to-date information on American Airlines' official website or contact their customer service for the latest details on checked bag fees.</b>

In [71]:
question = "What is the fee for checked bag for economy class to Cuba on American Airlines?"
query_response = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {query_response}</b>"))


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the fee for checked bag for economy class to Cuba on American Airlines?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_val

<b>Q: What is the fee for checked bag for economy class to Cuba on American Airlines?<b>

<b>A: The fee for a checked bag for economy class to Cuba on American Airlines is $150 for the 2nd bag, except during the seasonal period from November 16 to January 9 when the fee is $200.</b>

In [73]:
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer

chat_store = SimpleChatStore()

chat_memory = ChatMemoryBuffer.from_defaults(
    token_limit=3000,
    chat_store=chat_store,
    chat_store_key="user1",
)

chat_engine = kg_index.as_chat_engine(memory=chat_memory)

In [74]:
response = chat_engine.chat("What is the fee for a checked bag on American Airlines?")

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'What is the fee for a checked bag on American Airlines?'}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1, 'tool_choice': 'auto', 'tools': [{'type': 'function', 'function': {'name': 'query_engine_tool', 'description': 'Useful for running a natural language query\nagainst a knowledge base and get back a natural language response.\n', 'parameters': {'type': 'object', 'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input']}}}]}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_valu

In [75]:
display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {response}</b>"))


<b>Q: What is the fee for checked bag for economy class to Cuba on American Airlines?<b>

<b>A: The fee for a checked bag on American Airlines varies depending on the destination. For travel within or between the U.S., Puerto Rico, and U.S. Virgin Islands, the 1st checked bag fee is $40 ($35 if paid online). For travel to or from Canada, Caribbean, Mexico, Central America, and Guyana, the 1st checked bag fee is $35.</b>

In [76]:
question_2 = "I'm flying economy class to Cuba."
response_2 = chat_engine.chat(question_2)
display(Markdown(f"<b>Q: {question_2}<b>\n\n<b>A: {response_2}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'What is the fee for a checked bag on American Airlines?'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_Gd1fV21PI20IzDWbsRnRo0f3', 'function': {'arguments': '{"input":"What is the fee for a checked bag on American Airlines?"}', 'name': 'query_engine_tool'}, 'type': 'function'}]}, {'role': 'tool', 'content': 'The fee for a checked bag on American Airlines varies depending on the destination and when the ticket was issued. For travel within or between the U.S., Puerto Rico, and U.S. Virgin Islands, the 1st checked bag fee is $40 ($35 if paid online). For travel to or from Canada, Caribbean, Mexico, Central America, and Guyana, the 1st checked bag fee is $35.', 'name': 'query_engine_tool', 'tool_call_id': 'call_Gd1fV21PI20IzDWbsRnRo0f3'}, {'role': 'assistant', 'content': 'The fee for a checked bag on American

<b>Q: I'm flying economy class to Cuba.<b>

<b>A: The fee for a checked bag on American Airlines for economy class to Cuba is $150 for the 2nd bag, except for Main Plus tickets issued on/after September 20, 2023, which have an extra free checked bag.</b>

In [77]:
display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {response}</b>"))
display(Markdown(f"<b>Q: {question_2}<b>\n\n<b>A: {response_2}</b>"))

<b>Q: What is the fee for checked bag for economy class to Cuba on American Airlines?<b>

<b>A: The fee for a checked bag on American Airlines varies depending on the destination. For travel within or between the U.S., Puerto Rico, and U.S. Virgin Islands, the 1st checked bag fee is $40 ($35 if paid online). For travel to or from Canada, Caribbean, Mexico, Central America, and Guyana, the 1st checked bag fee is $35.</b>

<b>Q: I'm flying economy class to Cuba.<b>

<b>A: The fee for a checked bag on American Airlines for economy class to Cuba is $150 for the 2nd bag, except for Main Plus tickets issued on/after September 20, 2023, which have an extra free checked bag.</b>